In [31]:
import re
from collections import namedtuple

In [32]:
from typing import NamedTuple

In [33]:
import operator as op
from functools import partial

class Amounts(NamedTuple):
    ore: int = 0
    clay: int = 0
    obsidian: int = 0
    geode: int = 0
    
    def __add__(self, other: "Amounts"): return Amounts(*map(op.add,self,other))
    def __sub__(self, other: "Amounts"): return Amounts(*map(op.sub,self,other))
    def __mul__(self, other):
        if isinstance(other, Amounts):
            return Amounts(*map(op.mul,self,other))
        else:
            return Amounts(*map(partial(op.mul,other),self))
            


In [34]:
BP = namedtuple("Blueprint", "ore,clay,obsidian,geode")

In [103]:
def parse(fname):
    for line in open(fname):
        if line.strip():
            _,*ns= re.findall('[0-9]+', line)
            ns = map(int,ns)
            yield BP(ore=Amounts(ore=next(ns)), 
                               clay=Amounts(ore=next(ns)),
                               obsidian=Amounts(ore=next(ns),clay=next(ns)),
                                geode=Amounts(ore=next(ns),obsidian=next(ns)))
            
bps = list(parse("input"))
bps

[Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=4, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=4, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=16, geode=0)),
 Blueprint(ore=Amounts(ore=2, clay=0, obsidian=0, geode=0), clay=Amounts(ore=2, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=2, clay=15, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=7, geode=0)),
 Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=3, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=2, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=7, geode=0)),
 Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=3, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=2, clay=17, obsidian=0, geode=0), geode=Amounts(ore=3, clay=0, obsidian=16, geode=0)),
 Blueprint(ore=Amounts(ore=2, clay=0, obsidian=0, geode=0), clay=Amounts(ore=2, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore

In [95]:
def evaluate(moves: list, bp: BP, t=24):
    minerals = Amounts()
    robots = Amounts(ore=1)
    breakpoint()
    while t > 0:
        #build robots
        move = moves.pop(0) if len(moves) > 0 else None
        if move:
            for i in range(4):
                minerals = minerals - bp[i]*move[i]
            
            if any(m < 0 for m in minerals):
                raise ValueError(minerals)
            
            new_robots = robots + move
            
        #collect
        minerals = minerals + robots
        
        #complete robots
        if move:
            robots = new_robots
        
        print(24-t+1,minerals)
        t-=1
    return minerals

moves=[
    None,None,Amounts(clay=1),
    None,Amounts(clay=1),None,
    Amounts(clay=1),None,None,
    None,Amounts(obsidian=1),Amounts(clay=1),
    None,None,Amounts(obsidian=1),
    None,None,Amounts(geode=1),
    None,None,Amounts(geode=1),
    None,None,None
]
#evaluate(moves,bps[0])

In [96]:
def minerals_spent_building_robots(robots, bp):
    minerals = Amounts()
    for i in range(4):
        minerals = minerals + bp[i]*robots[i]   
    return minerals

def can_build_robots(robots, minerals, bp):
    minerals = minerals - minerals_spent_building_robots(robots,bp)
    return all(m >= 0 for m in minerals)


In [97]:

def valid_moves(robots, minerals, bp):    
    # just moves of one robot at a time for now
    for i in range(4):
        robot = [0,0,0,0]
        robot[i] = 1
        if can_build_robots(Amounts(*robot), minerals, bp):
            yield robot

list(valid_moves(Amounts(), Amounts(2),bps[0]))

[[0, 1, 0, 0]]

In [98]:
from math import ceil

def lower_bound(robots, minerals, bp, time_left):
    return minerals.geode + robots.geode*time_left

def upper_bound_1(robots, minerals, bp, time_left):
    # assuming we can build one geode robot per round until time_left given infinite resources
    # minerals.geode + robots.geode*time_left + (time_left-1) + (time_left-2) + (time_left -2) + ...
    # the series sum(i for i in range(m)) can be simplified to (m-1)*m/2)
    return minerals.geode + robots.geode*time_left + (time_left)*(time_left-1)//2

def upper_bound_2(robots, minerals, bp, time_left):
    ub = upper_bound_1(robots, minerals, bp, time_left)
    
    # unlike upper_bound_1, we don't assume infinite resources to build geode robots. We have to take
    # into account how much obsidian we can rely (now assuming infinite resources to build those)
    max_obsidian = minerals.obsidian + robots.obsidian * time_left + (time_left)*(time_left-1)//2
    
    # given that we have this max obsidian, the number of geode robots we can harvest may be smaller
    max_geode_robots = ceil(max_obsidian / bp.geode.obsidian) 
    
    # now, instead of computing the sum of the series time_left -> 0, we compute the sum 
    # from time_left -> (time_left - max_geode_robots)
    if max_geode_robots < time_left:
        ub = ub - (time_left - max_geode_robots)*(time_left-max_geode_robots+1) // 2
        
    return ub

upper_bound = upper_bound_2

In [91]:
assert sum(range(10)) == 10*9//2

In [104]:
from collections import deque

def solve(bp: BP, t=24):

    best = 0
    # state is a tuple of (robots, minerals, time_left)
    todo = deque([(Amounts(ore=1), Amounts(), t)])
    visited = {}    
    state = None
    it = 0
    while todo:
        it += 1
        robots, minerals, time_left = state = todo.pop()
        
        lb = lower_bound(robots, minerals, bp, time_left)
            
        best = max(best, lb)
        
        vs = (robots, minerals)
        if visited.get(vs,0) >= time_left:
            continue
        else:
            visited[vs] = time_left
            
        if (it % 100_000 == 0): print(it, len(todo), len(visited), time_left, best)

        if time_left == 0:
            continue
         
        if upper_bound(robots, minerals, bp, time_left) <= best:
            continue
        
        # add the do nothing move
        todo.append((robots, minerals + robots, time_left - 1))
        
        for move in valid_moves(robots, minerals, bp):
            new_state = (
                robots + move,
                minerals - minerals_spent_building_robots(move, bp) + robots,
                time_left - 1
            )
            todo.append(new_state)

        
    return best

In [105]:
%time solve(bps[1])

100000 20 83584 5 10
200000 19 150119 5 10
300000 24 210037 4 10
500000 20 317204 6 11
700000 21 417405 7 11
800000 18 472654 6 12
900000 21 507882 6 12
1300000 21 678223 7 15
1500000 18 717692 6 15
1600000 18 736003 6 15
1700000 10 763385 11 15
1800000 21 808768 6 15
CPU times: user 27.6 s, sys: 58 ms, total: 27.7 s
Wall time: 27.7 s


15

In [106]:
bps[0]

Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=4, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=4, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=16, geode=0))

In [107]:
2,7

(2, 7)

In [108]:
from tqdm.notebook import tqdm
def tenumerate(*args): return enumerate(tqdm(*args)) 

ans1 = sum((i+1)*solve(bp) for i,bp in tenumerate(bps))

  0%|          | 0/30 [00:00<?, ?it/s]

100000 20 83584 5 10
200000 19 150119 5 10
300000 24 210037 4 10
500000 20 317204 6 11
700000 21 417405 7 11
800000 18 472654 6 12
900000 21 507882 6 12
1300000 21 678223 7 15
1500000 18 717692 6 15
1600000 18 736003 6 15
1700000 10 763385 11 15
1800000 21 808768 6 15
100000 17 65144 4 5
100000 26 78503 1 4
200000 24 141763 4 5
300000 16 202704 6 5
500000 26 308879 5 5
600000 20 354264 4 5
800000 14 456048 5 5
1000000 36 549246 2 7
1100000 26 587277 5 7
1300000 14 635292 6 7
1500000 26 727029 5 7
1600000 29 762323 5 9
1800000 25 805816 5 9
1900000 23 816607 5 9
2000000 14 830299 8 9
2100000 8 849369 12 9
2200000 22 890256 6 9
200000 16 117743 5 2
300000 16 166172 3 2
500000 26 254726 4 3
600000 10 278555 10 3
800000 19 333609 5 4
1000000 15 401693 7 4
200000 20 119659 5 0
100000 24 68411 2 2
200000 19 123568 3 2
300000 18 176564 3 2
600000 17 317178 3 2
100000 23 77164 8 8
400000 18 255547 5 8
600000 26 379413 4 8
700000 25 429002 6 8
800000 24 488234 5 8
900000 22 553802 4 8
1000000 2

In [109]:
ans1

1150